In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import re
from tensorflow.keras.preprocessing.text import one_hot
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
import string
import sklearn
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df=pd.read_csv(r'IMDB Dataset.csv')
dataset=df[:10000]
toke=[]
def clean_text(dataset):
    clean_dataset=list()
    lines=dataset['review'].values.tolist()
    for text in lines:
        text=text.lower()
        pattern=re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text=pattern.sub('',text)
        text=re.sub(r"[,','.\"!@#$%^&*(){}?/;`~:<>+=-_]","",text)
        tokens=word_tokenize(text)
        # print(tokens)
        table=str.maketrans('','',string.punctuation)
        stripped=[w.translate(table)for w in tokens]
        words=[word for word in stripped if word.isalpha()]
        words=' '.join(words)
        clean_dataset.append(words)

    return clean_dataset
clean_dataset=clean_text(dataset)
wordnet= WordNetLemmatizer()
lemmetized_words=[]
for i in range(len(clean_dataset)):
    review = re.sub('[^a-zA-Z]', ' ', clean_dataset[i])
    review= review.split()
    review= [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    # review= ' '.join(review)
    lemmetized_words.append(review)
# print(lemmetized_words[:5])
for i in range(10):
  print(lemmetized_words[i])

vocab_size= 1000

['one', 'reviewer', 'mentioned', 'watching', 'oz', 'episode', 'youll', 'hooked', 'right', 'exactly', 'happened', 'mebr', 'br', 'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scene', 'violence', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'hearted', 'timid', 'show', 'pull', 'punch', 'regard', 'drug', 'sex', 'violence', 'hardcore', 'classic', 'use', 'wordbr', 'br', 'called', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focus', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inwards', 'privacy', 'high', 'agenda', 'em', 'city', 'home', 'manyaryans', 'muslim', 'gangsta', 'latino', 'christian', 'italian', 'irish', 'moreso', 'scuffle', 'death', 'stare', 'dodgy', 'dealing', 'shady', 'agreement', 'never', 'far', 'awaybr', 'br', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'go', 'show', 'wouldnt', 'dare', 'forget', 'pretty', 'picture', 'painted', 'mainstream', 'aud

In [ ]:
le= LabelEncoder()
xy= dataset['sentiment'].values.tolist()
le.fit(['positive', 'negative'])
new_xy=le.transform(xy)
print(new_xy)

[1 1 1 ... 0 0 1]


In [ ]:
import numpy as np
#splitting the Dataset into train and test set
totalRows=np.shape(lemmetized_words)[0]

splitRatio=0.75
splitPoint=int(splitRatio*totalRows)

trainReviews=lemmetized_words[:splitPoint]
trainLabels=new_xy[:splitPoint]
testReviews=lemmetized_words[splitPoint:]
testLabels=new_xy[splitPoint:]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,1))
output = vectorizer.fit_transform(lemmetized_words)
# tf_features_test = vectorizer.transform(imdb_test['review'])
# xd= output.toarray()
l=vectorizer.get_feature_names_out()

AttributeError: ignored

In [ ]:
#learning word embeddings on training data using Gensim library
from gensim.models import Word2Vec, KeyedVectors
import nltk

embeddingsSize=256
model=Word2Vec(lemmetized_words, size=embeddingsSize, window=5, min_count=1, workers=4)
######################Training of Word Embeddings Vector Completed#########
import numpy as np
def getVectors(dataset):
  singleDataItemEmbedding=np.zeros(embeddingsSize)
  vectors=[]
  for dataItem in dataset:
    wordCount=0
    for word in dataItem:
      if word in model.wv.vocab:
        singleDataItemEmbedding=singleDataItemEmbedding+model.wv[word]
        wordCount=wordCount+1
  
    singleDataItemEmbedding=singleDataItemEmbedding/wordCount  
    vectors.append(singleDataItemEmbedding)
  return vectors
# lemmetizedwordsgetvectors=getVectors(lemmetized_words)
trainReviewVectors=getVectors(trainReviews)
testReviewVectors=getVectors(testReviews)

In [ ]:
df= pd.DataFrame(lemmetizedwordsgetvectors)
df["sentiments"]=new_xy
df.to_csv('DATASET_REVISED.csv')

In [ ]:
df= pd.DataFrame(trainReviewVectors)
df["sentiments"]=trainLabels
df.to_csv('TRAIN_review.csv')

In [ ]:
df= pd.DataFrame(testReviewVectors)
df["sentiments"]=testLabels
df.to_csv('TEST_review.csv')

In [ ]:
from sklearn.metrics import accuracy_score
#add path of google drive to environment variable to load python files from google drive
import sys
# sys.path.insert(1, "/content/drive/My Drive/nlp assignments/assignment 4")
# from visualization import plot_confusion_matrix_from_data
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import f1_score

def printResults(y_true, y_predicted):
  print("Accuracy= ", accuracy_score(y_true, y_predicted))

  columns=['false', 'true']
  # plot_confusion_matrix_from_data(y_true, y_predicted, columns)

  precision, recall, fscore, support = score(y_true, y_predicted)

  print('###########################################')
  print('precision: {}'.format(precision))  
  print('recall: {}'.format(recall))
  print('fscore: {}'.format(fscore))
  print('support: {}'.format(support))
  print('###########################################3')

  print('Macro F1 ',f1_score(y_true, y_predicted, average='macro'))

  print('Micro F1 ', f1_score(y_true, y_predicted, average='micro'))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
clfNB = MultinomialNB()

scaler = MinMaxScaler()
scaledTrainX= scaler.fit_transform(trainReviewVectors)
scaledTestX = scaler.fit_transform(testReviewVectors)
clfNB.fit(scaledTrainX, trainLabels)

#test naive bayes accuracy
testLabelsPredicted=list(clfNB.predict(scaledTestX))

#print results
print("####################RESULTS OF NAIVE BAYES CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NAIVE BAYES CLASSIFIER##################
Accuracy=  0.654
###########################################
precision: [0.6004902  0.70532915]
recall: [0.66156616 0.64794816]
fscore: [0.62955032 0.67542214]
support: [1111 1389]
###########################################3
Macro F1  0.6524862300179582
Micro F1  0.654


In [ ]:
#neural network
from sklearn.neural_network import MLPClassifier

clfMLP = MLPClassifier(hidden_layer_sizes=(10, 10, 10))
clfMLP.fit(trainReviewVectors, trainLabels)
  
testLabelsPredicted=list(clfMLP.predict(testReviewVectors))

#print results
print("####################RESULTS OF NEURAL NETWORK CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NEURAL NETWORK CLASSIFIER##################
Accuracy=  0.7824
###########################################
precision: [0.75245098 0.81112853]
recall: [0.79259897 0.7735426 ]
fscore: [0.77200335 0.79188982]
support: [1162 1338]
###########################################3
Macro F1  0.7819465884581764
Micro F1  0.7824


In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

clfRF = RandomForestClassifier(n_estimators = 100)
# Train the model on training data
clfRF.fit(trainReviewVectors, trainLabels);

testLabelsPredicted=list(clfRF.predict(testReviewVectors))

#print results
print("####################RESULTS OF Random Forest CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF Random Forest CLASSIFIER##################
Accuracy=  0.7588
###########################################
precision: [0.74264706 0.77429467]
recall: [0.7593985  0.75825019]
fscore: [0.75092937 0.76618845]
support: [1197 1303]
###########################################3
Macro F1  0.7585589065817562
Micro F1  0.7588
